In [ ]:
import logging
import os

from src.ember.core.registry.model.settings import settings
from src.ember.core.registry.model.model_registry import ModelRegistry
from src.ember.core.registry.model.core.schemas.model_info import ModelInfo
from src.ember.core.registry.model.core.schemas.provider_info import ProviderInfo
from src.ember.core.registry.model.core.schemas.cost import ModelCost, RateLimit
from src.ember.core.registry.model.core.services.usage_service import UsageService

In [2]:
from src.ember.core.registry.model.core.services.model_service import ModelService

In [3]:
from src.ember.core.registry.model.model_enum import OpenAIModelEnum as OME

In [4]:
# Example of using pydantic-based config or environment variables
openai_key = settings.openai_api_key or os.getenv("OPENAI_API_KEY", "")

In [5]:
anthropic_key = settings.anthropic_api_key or os.getenv("ANTHROPIC_API_KEY", "")

In [6]:
deepmind_key = settings.google_api_key or os.getenv("GOOGLE_API_KEY", "")

In [9]:
registry = ModelRegistry()

In [10]:
usage_service = UsageService()

In [11]:
svc = ModelService(registry=registry, usage_service=usage_service)

In [ ]:
# Register an OpenAI GPT-4o model
openai_info = ModelInfo(
    model_id="openai:gpt-4o",
    model_name="gpt-4o",
    cost=ModelCost(input_cost_per_thousand=0.03, output_cost_per_thousand=0.06),
    rate_limit=RateLimit(tokens_per_minute=80000, requests_per_minute=5000),
    provider=ProviderInfo(name="OpenAI", default_api_key=openai_key),
    api_key=openai_key,
)
registry.register_model(openai_info)

In [ ]:
# 1) Call the model using a string ID
response = svc(OME.GPT_4O, "Hello world!")
print("String ID response:", response.data)
usage_summary = usage_service.get_usage_summary("openai:gpt-4")
print("Usage summary (string ID):", usage_summary)

In [ ]:
# 2) Call the model via the ModelService with an Enum
response = svc(OME.GPT_4O, "Hello again via enum!")
print("Enum response:", response.data)
usage_summary = usage_service.get_usage_summary(OME.GPT_4O.value)
print("Usage summary (enum):", usage_summary)

In [ ]:
# 3) Direct usage: retrieving the model and calling it as a function
gpt4_model = svc.get_model(OME.GPT_4O)
response = gpt4_model("What is the capital of France?")
print("Direct usage response:", response.data)

In [ ]:
# 4) Option 1 with enum repeated
response = svc(OME.GPT_4O.value, "What is the capital of Germany?")
print("Enum usage response data:", response.data)
print("Enum usage response usage:", response.usage)

In [ ]:
# 5) Option 2 with string repeated
response = svc("openai:gpt-4o", "What is the capital of Italy?")
print("String usage response data:", response.data)

In [ ]:
# 6) "Pytorch-like" usage (calling forward() explicitly)
response = gpt4_model("What is the capital of Spain?")
print("Forward() usage response:", response.data)